In [1]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# 近代数学とプログラミング

## 集合論と序数

* [5. Data Structures — Python 3.10.5 documentation](https://docs.python.org/3/tutorial/datastructures.html#sets)
  - [Built-in Types — Python 3.10.5 documentation](https://docs.python.org/3/library/stdtypes.html#set)

### Pythonの空集合 $\phi$ の扱い:

``list``と``tuple``と``dict``と``set``

In [2]:
[], list([]), list()

([], [], [])

In [3]:
list(map(type, ([], list([]), list())))

[list, list, list]

In [6]:
list(map(bool, ([], list([]), list())))

[False, False, False]

In [7]:
if not []:
    print('empty')

empty


In [9]:
(), tuple(()), tuple()

((), (), ())

In [10]:
list(map(type, ((), tuple(()), tuple())))

[tuple, tuple, tuple]

In [11]:
list(map(bool, ((), tuple(()), tuple())))

[False, False, False]

In [12]:
if not ():
    print('empty')

empty


In [13]:
{}, dict({}), dict()

({}, {}, {})

In [14]:
list(map(type, ({}, dict({}), dict())))

[dict, dict, dict]

In [15]:
list(map(bool, ({}, dict({}), dict())))

[False, False, False]

In [17]:
if not {}:
    print('empty')

empty


In [26]:
d={'jack': 4098, 'sape': 4139}
d

{'jack': 4098, 'sape': 4139}

In [27]:
type(d), d['jack']

(dict, 4098)

* [PEP 218 – Adding a Built-In Set Object Type | peps.python.org](https://peps.python.org/pep-0218/)

In [4]:
{}, set({}), set()

({}, set(), set())

In [4]:
list(map(type, ({}, set({}), set())))

[dict, set, set]

In [18]:
list(map(bool, ({}, set({}), set())))

[False, False, False]

In [19]:
if not set({}):
    print('empty')

empty


In [21]:
s={1,2,3}
s

{1, 2, 3}

In [25]:
type(s)

set

### von Neumannの序数

* (1903&ndash;1957) [John von Neumann - Wikipedia](https://en.wikipedia.org/wiki/John_von_Neumann)
  - [Ordinal number - Wikipedia](https://en.wikipedia.org/wiki/Ordinal_number#Von_Neumann_definition_of_ordinals)
    - [Set-theoretic definition of natural numbers - Wikipedia](https://en.wikipedia.org/wiki/Set-theoretic_definition_of_natural_numbers#:~:text=Definition%20as%20von%20Neumann%20ordinals,-See%20also%3A%20Zermelo&text=The%20set%20N%20of%20natural,Peano%20axioms%20%28Goldrei%201996%29.)

ひとつ前の集合を包含する:

$$
  \begin{align}
    0 &= \{\} \\
    S(a) &= a \cup \{a\} 
  \end{align}
$$

0. $\phi$
1. $\{\phi\}$
2. $\{\phi, \{\phi\}\}$
3. $\{\phi, \{\phi, \{\phi\}\}\}$

Pythonでは、集合は集合を含むことができない。

In [2]:
def succ(n):
    return n|{n}

In [3]:
succ(set())

TypeError: unhashable type: 'set'

In [4]:
set()|{set()}

TypeError: unhashable type: 'set'

In [5]:
0|{0}

TypeError: unsupported operand type(s) for |: 'int' and 'set'

次のように表現する:

In [6]:
n = [None]*10
n[0] = set()
n

[set(), None, None, None, None, None, None, None, None, None]

In [7]:
n[1] = {0}|n[0]
n

[set(), {0}, None, None, None, None, None, None, None, None]

In [8]:
n[2] = {1}|n[1]
n

[set(), {0}, {0, 1}, None, None, None, None, None, None, None]

In [9]:
n[3] = {2}|n[2]
n

[set(), {0}, {0, 1}, {0, 1, 2}, None, None, None, None, None, None]

In [10]:
n = [set()]

def succ(i):
    return {i}|n[i]

for i in range(10):
    n.append(succ(i))
n

[set(),
 {0},
 {0, 1},
 {0, 1, 2},
 {0, 1, 2, 3},
 {0, 1, 2, 3, 4},
 {0, 1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 3, 4, 5, 6, 7, 8},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}]

In [37]:
n[0], n[1]

(set(), {0})

集合の包含関係による順序が導入されているが、プログラミング的に応用は難しい･･･
* [Set (mathematics) - Wikipedia](https://en.wikipedia.org/wiki/Set_%28mathematics%29#Subsets)

$\{0,1,2\} \subset \{0,1,2,3,4\}$

In [134]:
type(5), type(3)

(int, int)

In [135]:
5<3, 5>3

(False, True)

In [136]:
n[5], n[3]

({0, 1, 2, 3, 4}, {0, 1, 2})

In [137]:
type(n[5]), type(n[3])

(set, set)

In [138]:
n[5]<n[3], n[5]>n[3]

(False, True)

In [139]:
n[5].issubset(n[3]), n[3].issubset(n[5])

(False, True)

In [140]:
(n[5]&n[3])==n[5], (n[5]&n[3])==n[3]

(False, True)

In [141]:
n[5]&n[3], n[5]|n[3], n[5]^n[3], n[5]-n[3]

({0, 1, 2}, {0, 1, 2, 3, 4}, {3, 4}, {3, 4})

In [54]:
list(map(len, n))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [38]:
sorted(n)

[set(),
 {0},
 {0, 1},
 {0, 1, 2},
 {0, 1, 2, 3},
 {0, 1, 2, 3, 4},
 {0, 1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 3, 4, 5, 6, 7, 8},
 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}]

In [60]:
sorted(n, key=lambda x: len(x))

[set(),
 {0},
 {0, 1},
 {0, 1, 2},
 {0, 1, 2, 3},
 {0, 1, 2, 3, 4},
 {0, 1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4, 5, 6},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 3, 4, 5, 6, 7, 8}]

In [59]:
sorted(n, key=lambda x: -len(x))

[{0, 1, 2, 3, 4, 5, 6, 7, 8},
 {0, 1, 2, 3, 4, 5, 6, 7},
 {0, 1, 2, 3, 4, 5, 6},
 {0, 1, 2, 3, 4, 5},
 {0, 1, 2, 3, 4},
 {0, 1, 2, 3},
 {0, 1, 2},
 {0, 1},
 {0},
 set()]

課題-2) 上で定義した自然数を表す集合のリスト``n``の要素の大小を判定する ``less_than()`` を定義せよ。

In [11]:
def less_than(n0, n1):
    ### BEGIN SOLUTION
    return n0<n1
    ### END SOLUTION

In [12]:
less_than(n[3], n[5])

True

In [13]:
less_than(n[5], n[3])

False

In [14]:
less_than(3, 5)

True

In [ ]:
---
以下、採点用のセルにつき編集できない:

In [ ]:
from nose.tools import assert_equal, assert_true

In [ ]:
assert_equal(less_than(n[3], n[5]), True)